In [ ]:
!pip install scikit-learn

In [ ]:
import pandas as pd

# URL of the CSV file
csv_url = 'https://raw.githubusercontent.com/Joshi-Aryan/news-category-classification/main/news_corpus(cleaned).csv'  # Replace with the actual URL of your CSV file

# Load the CSV file into a DataFrame and ignore the original index
combined_data = pd.read_csv(csv_url)

# Reset the index to be continuous and ignore the original index
combined_data.reset_index(drop=True, inplace=True)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from scipy import sparse

# Define a function to read data in chunks
def read_csv_in_chunks(url, chunk_size=1000):
    reader = pd.read_csv(url, chunksize=chunk_size)
    for chunk in reader:
        yield chunk

combined_data['headline'] = combined_data['headline'].fillna('')

# Convert the linguistic features to a sparse matrix
combined_data['text_length'] = combined_data['headline'].apply(len)
combined_data['num_words'] = combined_data['headline'].apply(lambda x: len(x.split()))
linguistic_features = combined_data[['text_length', 'num_words']]

combined_data = combined_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the data into train and test
X_train_text, X_test_text, y_train, y_test = train_test_split(combined_data['headline'], combined_data['category'], test_size=0.2, random_state=42)

# Character n-gram features
char_vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 5), lowercase=True)
char_ngram_features = char_vectorizer.fit_transform(X_train_text)

# Word n-gram features
word_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), lowercase=True)
word_ngram_features = word_vectorizer.fit_transform(X_train_text)

# Convert features to sparse matrices
char_ngram_features = sparse.csr_matrix(char_ngram_features)
word_ngram_features = sparse.csr_matrix(word_ngram_features)

# Combine the sparse matrices
combined_features = sparse.hstack((char_ngram_features, word_ngram_features))

# Release the memory occupied by X_train_text, char_ngram_features, and word_ngram_features
del X_train_text, char_ngram_features, word_ngram_features

linguistic_features = sparse.csr_matrix(linguistic_features)

# Ensure linguistic_features and combined_features have compatible dimensions
if linguistic_features.shape[0] != combined_features.shape[0]:
    linguistic_features = linguistic_features[:combined_features.shape[0], :]

combined_features = sparse.hstack((combined_features, linguistic_features)).tocsr()

'''# Release the memory occupied by linguistic_features
del linguistic_features
'''

'# Release the memory occupied by linguistic_features\ndel linguistic_features\n'

In [ ]:

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(combined_features, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
# Transform the test data
X_test_char_ngram_features = char_vectorizer.transform(X_test_text)
X_test_word_ngram_features = word_vectorizer.transform(X_test_text)

# Convert test features to sparse matrices
X_test_char_ngram_features = sparse.csr_matrix(X_test_char_ngram_features)
X_test_word_ngram_features = sparse.csr_matrix(X_test_word_ngram_features)

# Combine the test features
X_test_combined_features = sparse.hstack((X_test_char_ngram_features, X_test_word_ngram_features))

# Convert linguistic features of test data to sparse matrix
linguistic_features_test = pd.DataFrame({'text_length': X_test_text.apply(len), 'num_words': X_test_text.apply(lambda x: len(x.split()))})
linguistic_features_test = sparse.csr_matrix(linguistic_features_test)

# Ensure linguistic_features_test and X_test_combined_features have compatible dimensions
if linguistic_features_test.shape[0] != X_test_combined_features.shape[0]:
    linguistic_features_test = linguistic_features_test[:X_test_combined_features.shape[0], :]

X_test_combined_features = sparse.hstack((X_test_combined_features, linguistic_features_test)).tocsr()

# Release the memory occupied by X_test_combined_features and linguistic_features_test
'''del X_test_combined_features, linguistic_features_test
'''

'del X_test_combined_features, linguistic_features_test\n'

In [ ]:
y_pred = model.predict(X_test_combined_features)


print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7923408845738943
Classification Report:
               precision    recall  f1-score   support

  declarative       0.76      0.95      0.84       941
  exclamatory       1.00      0.31      0.47        13
         frag       0.87      0.84      0.85       609
   imperative       0.50      0.11      0.17        19
interrogative       0.82      0.12      0.20        77
      np_drop       0.77      0.43      0.55        79
          qws       0.83      0.09      0.16       116

     accuracy                           0.79      1854
    macro avg       0.79      0.41      0.47      1854
 weighted avg       0.80      0.79      0.76      1854

